In [1]:
import sys
sys.path.append("..")


In [2]:
import pandas as pd
from agents.physical_agent import PhysicalAgent

df = pd.read_csv("../data/raw/diabetes.csv")

physical_agent = PhysicalAgent()
physical_output = physical_agent.preprocess(df)

physical_output.head()


,BloodPressure,BMI,SkinThickness,BloodPressure_missing,BMI_missing,SkinThickness_missing
0,72.0,33.6,35.0,0,0,0
1,66.0,26.6,29.0,0,0,0
2,64.0,23.3,29.0,0,0,1
3,66.0,28.1,23.0,0,0,0
4,40.0,43.1,35.0,0,0,0


In [3]:
physical_output.shape
physical_output.isna().sum()


BloodPressure            0
BMI                      0
SkinThickness            0
BloodPressure_missing    0
BMI_missing              0
SkinThickness_missing    0
dtype: int64

In [4]:
metadata = physical_agent.get_metadata()

metadata


PhysicalAgentMetadata(features_processed=['BloodPressure', 'BMI', 'SkinThickness'], original_zero_counts={'BloodPressure': 35, 'BMI': 11, 'SkinThickness': 227}, imputation_medians={'BloodPressure': 72.0, 'BMI': 32.3, 'SkinThickness': 29.0}, imputed_counts={'BloodPressure': 35, 'BMI': 11, 'SkinThickness': 227})

TO TEST LAB AGENT

In [5]:
from agents.lab_agent import LabAgent

lab_agent = LabAgent()
lab_output = lab_agent.preprocess(df)

lab_metadata = lab_agent.get_metadata()

lab_output.head(), lab_metadata


(   Glucose  Insulin  Insulin_missing
 0    148.0    125.0                1
 1     85.0    125.0                1
 2    183.0    125.0                1
 3     89.0     94.0                0
 4    137.0    168.0                0,
 LabAgentMetadata(features_processed=['Glucose', 'Insulin'], original_zero_counts={'Glucose': 5, 'Insulin': 374}, imputation_medians={'Glucose': 117.0, 'Insulin': 125.0}, imputed_counts={'Glucose': 5, 'Insulin': 374}))

Lab Agent test

In [11]:
from agents.demographic_agent import DemographicAgent

demo_agent = DemographicAgent()
demo_output = demo_agent.preprocess(df)
demo_metadata = demo_agent.get_metadata()

demo_output.head(), demo_metadata


ImportError: cannot import name 'DemographicAgent' from 'agents.demographic_agent' (c:\Users\srira\AgentXHealth\notebooks\..\agents\demographic_agent.py)